In [2]:
import pandas as pandas
import datetime
import pickle
import os
os.chdir('..')

In [ ]:
import pickle
with open(r'data/flight_data_500K.pickle','rb') as flight:
     df_flight = pickle.load(flight)
with open(r'data/weather_dict.pickle','rb') as weather_data_file:
     df_weather = pickle.load(weather_data_file)

In [ ]:
df_flight['origin_state'] = df_flight['origin_city_name'].str[-2:]
df_flight['dest_state'] = df_flight['dest_city_name'].str[-2:]
df_flight.fl_date = pd.to_datetime(df_flight.fl_date)

In [ ]:
t = pd.json_normalize(df_weather).apply(lambda x : x[0]['TCAT'])
p = pd.json_normalize(df_weather).apply(lambda x : x[0]['PCAT'])

In [ ]:
t = t.T.reset_index().set_axis(['TCAT' for _ in range(t.shape[1])], axis = 0).T
t.columns = [t.columns, t.iloc[0]]
t = t[1:]

In [ ]:
p = p.T.reset_index().set_axis(['PCAT' for _ in range(p.shape[1])], axis = 0).T
p.columns = [p.columns, p.iloc[0]]
p = p[1:]

In [ ]:
t = t.merge(p, left_index=True, right_index=True)
a = (df_flight.fl_date.apply(lambda x : (x - datetime.timedelta(weeks = 1)).strftime('%Y%m%d')))
c = a[0:1].apply(lambda x : t.iloc[:, t.columns.get_level_values(1)==x])[0].merge(df_flight, left_index = True, right_on='origin_state')
c.columns = ['origin_TCAT', 'origin_PCAT'] + c.columns[2:].to_list()

In [ ]:
c = a[0:1].apply(lambda x : t.iloc[:, t.columns.get_level_values(1)==x])[0].merge(c, right_on='dest_state', left_index=True)
c.columns = ['dest_TCAT', 'dest_PCAT', 'origin_TCAT', 'origin_PCAT'] + c.columns[4:].to_list()
c

In [ ]:
import pickle
flight_data_engineered6 = c[['dest_TCAT', 'dest_PCAT', 'origin_TCAT', 'origin_PCAT','arr_delay']]
with open(r'data/flight_data_engineered6.pickle','xb') as weather_data_file:
     pickle.dump(flight_data_engineered6, weather_data_file)